In [1]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=df0f4ca33d3078031a7ab6a3f6c0cf4faf9447aacbb83ba3769f60d222ff623b
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install pysbd

In [48]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import torch
import pysbd
import re

In [3]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
df = pd.read_csv('/content/drive/MyDrive/ai_searchbar/bloomberg_data_telegram_15_03_2023.csv')

In [6]:
df.isnull().sum()

sender_id     0
message_id    0
text          9
date          0
photo         0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
# removing links
df['text_without_links'] = df['text'].apply(lambda x: re.sub(u'http(s|):\/\/\w+.\w+.*', '', x))

<ipython-input-8-b1007fdbb7b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_without_links'] = df['text'].apply(lambda x: re.sub(u'http(s|):\/\/\w+.\w+.*', '', x))


In [9]:
df.text_without_links = df.text_without_links.str.replace('\n', ' ')
df.text_without_links = df.text_without_links.str.replace('\xa0', '')

In [10]:
# full_corpus = df.text_without_links.sum()

In [11]:
df.text_without_links

0       **Asia's Richest:** A reversal in the extraord...
1       US prosecutors were investigating Signature Ba...
2       Biden plans to hold a phone call with Chinese ...
3       Everything Everywhere All at Once” wrapped its...
4       LIVE BLOG: The consequences of the collapse of...
                              ...                        
1608    📈 Why did the U.S. repo market get so bad, so ...
1609    🇭🇰 Hundreds of thousands of protesters marched...
1610    China is breeding mutant pigs that could help ...
1611    Hi, and a warm welcome to all of you who joine...
1613    Hello and welcome to Bloomberg's Telegram chan...
Name: text_without_links, Length: 1606, dtype: object

In [12]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [13]:
df.text_without_links = df.text_without_links.apply(lambda x: remove_emojis(x))

In [14]:
df[df.text_without_links.str.contains('http')]

,sender_id,message_id,text,date,photo,text_without_links


In [17]:
seg = pysbd.Segmenter(language="en", clean=False)

In [18]:
messages = list(zip(df.text_without_links, df.message_id))

In [19]:
sentenced = list(map(lambda x: [seg.segment(x[0]), x[1]], messages ))

In [20]:
ids_ = list(map(lambda x: np.repeat(x[1], len(x[0])).tolist(), sentenced))

In [21]:
ids_

[[1627, 1627, 1627, 1627, 1627],
 [1626, 1626],
 [1625, 1625, 1625, 1625, 1625],
 [1624, 1624],
 [1623, 1623],
 [1622, 1622, 1622],
 [1619, 1619, 1619],
 [1616, 1616],
 [1615, 1615],
 [1614, 1614],
 [1613],
 [1612],
 [1611, 1611, 1611, 1611],
 [1610, 1610, 1610],
 [1609, 1609, 1609, 1609],
 [1608, 1608],
 [1607, 1607],
 [1606, 1606],
 [1605, 1605, 1605],
 [1604, 1604, 1604],
 [1603, 1603],
 [1602, 1602, 1602],
 [1601, 1601],
 [1600, 1600, 1600],
 [1599, 1599],
 [1598, 1598, 1598, 1598],
 [1597, 1597, 1597, 1597],
 [1596, 1596, 1596, 1596],
 [1595, 1595, 1595, 1595, 1595],
 [1594, 1594, 1594],
 [1593, 1593, 1593, 1593],
 [1592, 1592],
 [1591, 1591],
 [1590, 1590],
 [1589, 1589, 1589, 1589],
 [1588, 1588, 1588, 1588, 1588],
 [1587, 1587, 1587],
 [1586, 1586, 1586],
 [1585, 1585, 1585],
 [1584, 1584, 1584, 1584],
 [1583, 1583, 1583, 1583],
 [1582, 1582, 1582, 1582],
 [1581, 1581, 1581],
 [1580, 1580, 1580],
 [1579, 1579, 1579, 1579],
 [1578, 1578, 1578, 1578],
 [1577, 1577, 1577, 1577, 15

In [22]:
flattened_ids = [item for sublist in ids_ for item in sublist]

In [23]:
sentences = [item for sublist in (map(lambda x: x[0], sentenced)) for item in sublist]

In [24]:
len(sentences) == len(flattened_ids)

True

In [25]:
sentences = pd.DataFrame({'sentences': sentences, 
              'id': flattened_ids})

In [35]:
sentences.sentences = sentences.sentences.str.strip()

In [36]:
sentences.to_csv('/content/drive/MyDrive/ai_searchbar/sentences.csv', index = False)

In [37]:
sentences.sample(3)

,sentences,id
5048,Hello!,325
5528,Beating the coronavirus faces long odds.,183
5368,Shuttered studios.,232


In [38]:
sentence_encoding = []
encoded_messages = model.encode(sentences.sentences)
np.save("/content/drive/MyDrive/ai_searchbar/messages_encoded.npy", encoded_messages)

In [39]:
encoded_messages = np.load('/content/drive/MyDrive/ai_searchbar/messages_encoded.npy')

In [40]:
encoded_messages

array([[-0.0313795 ,  0.03717247, -0.01005607, ..., -0.08800688,
        -0.00557959,  0.04324508],
       [ 0.06892988, -0.00070536,  0.00402268, ..., -0.06011203,
         0.03234569,  0.03489451],
       [-0.06116355,  0.01196444, -0.0044553 , ..., -0.05543737,
         0.01587115,  0.03624612],
       ...,
       [-0.07122789, -0.01519739,  0.00965715, ...,  0.07249122,
         0.07532555, -0.05834132],
       [ 0.03690844, -0.06305707,  0.05771654, ...,  0.00500402,
        -0.16226906,  0.00866578],
       [-0.03406273, -0.05368785,  0.00661559, ...,  0.03506849,
        -0.0282711 ,  0.05754004]], dtype=float32)

In [41]:
search = 'When does biden plan to meet with chinese leader?'
similarity = torch.nn.functional.cosine_similarity(torch.from_numpy(np.array(encoded_messages)), 
                                                    torch.from_numpy(model.encode(search)), dim=-1)

In [42]:
sentences.iloc[np.array(similarity.argsort(descending=True))[:3]]['sentences'].values

array(['**Here’s what we are following right now:**   US President Joe Biden will try to prevent ties with China from worsening further when he meets Xi Jinping on Monday, the first in-person meeting between the two world leaders since the pandemic started.',
       'Joe Biden says he expects to speak to Xi Jinping “within the next 10 days,” as the US considers lifting some tariffs on Chinese imports.',
       "President Biden will speak with Chinese leader Xi on Thursday as Nancy Pelosi's possible Taiwan visit stokes tensions."],
      dtype=object)

In [43]:
np.array(similarity.argsort(descending=True))[:3].tolist()

[121, 475, 452]

In [47]:
sentences.iloc[np.array(similarity.argsort(descending=True))[:3].tolist()].sentences.values

array(['**Here’s what we are following right now:**   US President Joe Biden will try to prevent ties with China from worsening further when he meets Xi Jinping on Monday, the first in-person meeting between the two world leaders since the pandemic started.',
       'Joe Biden says he expects to speak to Xi Jinping “within the next 10 days,” as the US considers lifting some tariffs on Chinese imports.',
       "President Biden will speak with Chinese leader Xi on Thursday as Nancy Pelosi's possible Taiwan visit stokes tensions."],
      dtype=object)